In [1]:
!pip install soynlp

In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt, Mecab
from soynlp.word import WordExtractor

# 데이터
- 전처리 및 맞춤법 검사가 된 문장

In [4]:
data = pd.read_csv('/aiffel/train_1000.csv')
data.head()

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화


In [164]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer
vocab_size = 30000

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

sentences = [tokenizer.tokenize(sen) for sen in sentences]
tokenizer_tf = Tokenizer(num_words=vocab_size)
tokenizer_tf.fit_on_texts(sentences)
word_dic = tokenizer_tf.word_index
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)
np.shape(padded)

training was done. used memory 3.219 Gbry 3.205 Gb
all cohesion probabilities was computed. # words = 1012
all branching entropies was computed # words = 26160
all accessor variety was computed # words = 26160


(4913, 433)

In [165]:
len(word_dic)

52616

In [166]:
train_data = padded
train_label = data['class']
print(len(train_data), len(train_label))

4913 4913


In [167]:
labels = {'직장 내 괴롭힘 대화':0, '기타 괴롭힘 대화':1, '갈취 대화':2, '협박 대화':3, '일반 대화': 4}
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3930 491 492
3930 491 492


# 모델 
- LSTM 사용

In [187]:
word_vector_dim = 1024
labels_size = len(labels)
hidden_size = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size//2))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(labels_size, activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 1024)        30720000  
_________________________________________________________________
lstm_24 (LSTM)               (None, None, 128)         590336    
_________________________________________________________________
lstm_25 (LSTM)               (None, None, 128)         131584    
_________________________________________________________________
lstm_26 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_16 (Dense)             (None, 30000)             1950000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 30000)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 5)                

In [188]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
              
epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
16/16 [==============================] - 8s 283ms/step - loss: 1.3063 - accuracy: 0.4443 - val_loss: 0.8029 - val_accuracy: 0.6741
Epoch 2/10
16/16 [==============================] - 4s 223ms/step - loss: 0.5444 - accuracy: 0.7718 - val_loss: 0.6236 - val_accuracy: 0.7678
Epoch 3/10
16/16 [==============================] - 4s 230ms/step - loss: 0.1780 - accuracy: 0.9405 - val_loss: 0.8120 - val_accuracy: 0.7943
Epoch 4/10
16/16 [==============================] - 4s 227ms/step - loss: 0.0754 - accuracy: 0.9743 - val_loss: 0.6889 - val_accuracy: 0.8350
Epoch 5/10
16/16 [==============================] - 4s 228ms/step - loss: 0.0341 - accuracy: 0.9896 - val_loss: 0.7739 - val_accuracy: 0.8269
Epoch 6/10
16/16 [==============================] - 4s 232ms/step - loss: 0.0080 - accuracy: 0.9982 - val_loss: 0.8070 - val_accuracy: 0.8310
Epoch 7/10
16/16 [==============================] - 4s 235ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.9961 - val_accuracy: 0.8187
Epoch 

In [189]:
pred = model.predict(test_X)

from sklearn.metrics import classification_report

pred_label = []
test_label = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.80      0.81      0.81       110
           1       0.75      0.73      0.74       115
           2       0.79      0.81      0.80        94
           3       0.77      0.81      0.79        69
           4       0.96      0.92      0.94       104

    accuracy                           0.82       492
   macro avg       0.81      0.82      0.82       492
weighted avg       0.82      0.82      0.82       492



In [190]:
to_text = {0:'직장 내 괴롭힘 대화', 1:'기타 괴롭힘 대화', 2:'갈취 대화', 3:'협박 대화', 4:'일반 대화'}
result = pd.DataFrame([pred_label,test_label]).T
error_target = result[result[0] != result[1]].reset_index(drop=True)
idx = result[result[0] != result[1]].index
error = data.non_label_sentence[idx]
idx = pd.DataFrame(zip(idx))
error = pd.DataFrame(zip(error))
error_result = pd.concat([idx,error,error_target], axis=1)
error_result.columns = ['idx', 'conversation', 'pred', 'real']
error_result['pred'] = error_result['pred'].apply(lambda x: to_text[int(x)])
error_result['real'] = error_result['real'].apply(lambda x: to_text[int(x)])

In [191]:
Sam = error_result[error_result['real'] == '직장 내 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
51,238,일반 대화,직장 내 괴롭힘 대화
4,13,기타 괴롭힘 대화,직장 내 괴롭힘 대화


In [192]:
Sam['conversation'][id[0]]

'니는 자동차 사고 싶은 생각 없나\n사고 싶죠 하하\n차 있으면 편하니까요\n나는 요즘 큰 가 너무 사고 싶어\n좋죠 하하\n운전하기도 좀 더 편하고 하하\n강아지도 데리고 다니고 캠핑도 가고 그러려면 큰 차가 필요한 것 같더라고\n오호 캠핑까지 생각하면 가 진짜 좋죠 하하\n근데 큰 차는 주차가 쉽지 않아서\n맞아요\n운전할 땐 좋은데 주차할 때 제약이 많아요\n서울은 진짜 주차 공간이 너무 부족해서 큰일이야\n아 그렇겠네요\n서울은 원룸 주차장도 돈 내야 하지 않아요\n어디 가려면 주차 공간 있는지부터 확인하고 나서야 한다\n주차장 없으면 그냥 뚜벅이가 더 나을 때가 있죠\n그렇지\n관리비에 주차비도 항상 포함되어 있었던 것 같다\n아 관리비 얼마 정도 나와요\n아 지금 우리 건물을 1대는 무료라\n관리비 2만 원 내고 있어\n오호 괜찮은데요\n1대 무료에 관리비도 안 비싸네요 하하\n진짜 감사하다'

In [193]:
Sam['conversation'][id[1]]

'이번에 3차 접종인가 나온 거 알아\n3차 접종 까지해\n몰랐어\n그 모더나는 아니고 화이자 3차도 있다더라\n아 나 화이자 맞았는데 그럼 한번 더 맞아야 돼\n응 아마도\n너 곧 부스터샷이라고 맞을 거야\n아 너무 싫다\n2차도 맞기 싫은 거 맞은 건데\n그치\n근데 맞으라니까 어쩔 수 없지\n하긴 진짜 이게 무슨 민주주위니 하하\n너 화이자 맞았었어\n난 기억도 안 났네\n너는 뭐 맞았는데\n그거 볼 수 있잖아\n나 아마 모더나일 거야\n모더나는 3차 접종 없다더라\n나이스\n와 부럽다 진짜\n나도 모더나 맞을걸\n이미 늦었잖아\n그냥 3차 후기 한 번 나중에 읽어봐\n나중에 우리나라 맞으면 후기 보고 맞을지 결정해야지\n응 아마 곧 너한테 더 와 닿을 거야\n진짜 안맞길 조심히 바래본다'

In [194]:
Sam = error_result[error_result['real'] == '기타 괴롭힘 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
54,257,갈취 대화,기타 괴롭힘 대화
34,155,갈취 대화,기타 괴롭힘 대화


In [195]:
Sam['conversation'][id[0]]

'오빠가 너무 웃겨서 같이 놀면 재밌을 것 같아요\n나중에 여행도 같이 가요\n여행은 너 남친 생기면 생각해보자\n어 오 그거 좋은 생각 같아요\n아니 우리 셋만\n아들 포함 시키지 말아줘\n이박 삼일이라도 넘 좋을 것 같은데요\n그러게\n생각만 해도 신나는데\n하하 우리에게 자유를 달라 더 열일 해야겠네요\n나도 육아 더 열심히 하고 있어야겠네\n언제가 될라나 그 날이\n하하 셋이 호캉스라도 가야겠어요\n특가 나오면 공유할게요\n좋아 좋아\n나 호캉스 안 가봤다구\n하하 좋아요 좋아요 아주 그냥 가서 먹고 놀고 수다 떨고 재밌을 것 같아요\n호캉스는 안 해봤어도 조식은 좋아합니다\n하하 너무 좋아요\n우리 열심히 돈 모아서 가요\n나 조금씩 모아볼게 우리의 호캉스를 위하여\n하하 힐링을 위해 조금 참고 모아 봅니다 하하\n제주도도 셋이 갈 수 있을까'

In [196]:
Sam['conversation'][id[1]]

'역시 치킨은 진리라니께\n맛나게 묵어브럿냐\n치킨은 왜 안 질릴까\n나도 치킨은 안 질리드라\n엉 역시 치킨은 언제나 맛있어\n가성비도 좋고 맛도 한결같아\n마자 치킨 값 오르면 빡칠 듯\n그니까 근데 한번 오르지 않았냐\n치킨은 서민의 음식이야\n너무 비싸져서 서민의 음식은\n야 떡볶이 값 실화냐\n얼마냐 떡볶이\n떡볶이 안 시켜 먹냐\n떡볶이는 집에서 해 먹는 레알 서민 음식\n기본이 2만 원임\n추가하면 노답임\n장난하나\n떡볶이 2만 원 웬 말임'

In [197]:
Sam = error_result[error_result['real'] == '갈취 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
41,193,협박 대화,갈취 대화
24,107,기타 괴롭힘 대화,갈취 대화


In [198]:
Sam['conversation'][id[0]]

'그건 맞지\n근데 난 집은 층간 소음이 잘 돼야 해\n층간 소음을 유발하는 것들은 양해를 구해야 해\n아니 모르겠는데\n잘 안 살았으면 좋겠어\n밑에 집 날 너무 화나게 해\n가끔 연락이 오나 봐 아직도\n맞아\n가끔 연락 오는데 내가 집에서 아무것도 안 하고 있거나 집에 없을 때도 시끄럽다고 연락 와서 집 아니라고 해\n소음공해 방지를 위해 밑에다 뭐 깔았잖아\n그니까 진짜 화난다니까\n다음에는 층간 소음 진짜 잘 되는 집 갈 거야\n층간 소음을 방지하는 방법은 뭐가 있지\n집에서 조금의 대화도 없는 게 말이 돼\n가족끼리 최소한의 대화가 없는 건 말이 안 돼\n맞아\n아 진짜 이 집 구할 때도 다른 건 됐고 층간 소음 없는 집 달라고 구한 건데\n우리는 가족이라는 공동체 안에 있어'

In [199]:
Sam['conversation'][id[1]]

'미국에 피자 사러 갔다가 5000억 원 복권 당첨된 사람 나왔단다\n헐 와 진심 부럽다 5000억\n피자 먹으러 갔다가 돈벼락 맞았네\n될놈될인갑다\n내가 원하는 곳에 초호화스럽게 엄빠집 짓고 내 집 짓고 차 사고 해도 남네\n메가밀리언 복권이라는데\n우리나라 로또 같은 건 갑다\n동생들 집도 다 사주고 건물 하나 사서 월세 받으면 살면 딱이겠다 하하\n이런 상상으로 대리만족\n피자가게 주인도 당첨 수수료로 1200만원 받았대\n우리나라 로또는 금액이 오히려 작은 편이더라구\n다른 나라는 걸리면 몇 대가 먹고 살 듯\n진짜 부럽다\n복권 사러 명당 찾아다녀도 나는 안 되던데\n그러게 될놈될이라는 거지\n제발 로또 한 번만 되라 하하\n피자가게 가서 복권 샀는데 5000억이 웬 말이고\n그니까 부럽고 또 부럽다\n이번에 대구 가서도 로또 명당이 있어서 로또 사왔는데 본전이라도 찾길\n그저 부럽네\n난 언제 걸릴꼬 으이\n이번 생은 어렵고 그런 건 아니겠제\n그건 제발\n이번 생에 최대한 빨리 지금'

In [200]:
Sam = error_result[error_result['real'] == '협박 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
37,168,직장 내 괴롭힘 대화,협박 대화
55,261,기타 괴롭힘 대화,협박 대화


In [201]:
Sam['conversation'][id[0]]

'나 모발 이식하고 싶당\n머리 빠지나 나도\n엠자 이마 넘 짜증\n머리가 점점 가늘어지네\n했던데 나더라 다시 신기\n아 닌 엠이가 난 정수리가\n정수리는 못 심지 않음\n그거 말고 뭐 쓰는 거 있던데\n몰라 얼만지\n모내기하듯이 심던데\n아 진짜가 모름\n못 심나\n얌 정수리는 그거 해라 타투\n비싸던데 진짜\n타투 하니까 감쪽같더라\n정수리 타투는 가능하나 근데\n정수리 타투 있음\n그래 나 요새 머리 넘 가늘어서'

In [202]:
Sam['conversation'][id[1]]

'가을 옷 사야 하는데 괜찮은 쇼핑몰 알아\n난 거의 네이버쇼핑에서 많이 보는데 요즘 이쁜 거 많이 나왔드라\n난 지그재그로 많이 보는데 쇼핑몰 다 모아 놔서 좋긴 한데 너무 많아서 고르기 힘들 때도 있음\n맞어 요즘 그렇게 모든 쇼핑몰 모아서 보여주는 플랫폼들이 있어서 품목 고르는 게 좋긴 한 거 같애\n하 가을 옷을 살지 가을 빨리 지나가니까 좀 기다릴지 고민이네\n난 좀 기다리는 거 추천\n오히려 가을에는 가디건이나 자켓류로 사는 것도 괜찮은 거 같애\n요즘 쇼핑몰 엄청 할인하길래\n가격이 다 싸더라고\n충동구매 욕구 뿜뿜이야\n맞어 가을이 너무 짧아서 할인한다고 넘어가면 안돼\n영양제는 아직이야\n옷이 사고 싶지만 너희들 말 듣고 참아봐야지\n아이허브는 어떤 사이트인데 애견용품까지 판매해\n아이허브 해외 영양제 제품 파는 직구 쇼핑몰인데\n오 해외영양제 좋다\n나도 한번 들어가서 구경 해봐야징\n반려동물 영양제 간식 이런 것도 팔더라고\n내가 영양제 살 때 거기서 사\n오 안 그래도 비타민 떨어져서 찾아보고 있었는데 아이허브 구경해야겠다\n배송이 빠른가\n요즘 해외도 빠르지\n아무래도 국내처럼 빠르지는 않고\n그리고 영양제 6병 까지만 돼\n수입신고 할 때 제한되어 있더라고\n아 해외배송이라 그런 게 있구나\n해외 직구는 한번도 안 해봐서 요번에 한번 해봐야겠당'

In [203]:
Sam = error_result[error_result['real'] == '일반 대화'].sample(n=2)
id = Sam.index
Sam[['idx','pred', 'real']]

,idx,pred,real
19,86,갈취 대화,일반 대화
13,62,직장 내 괴롭힘 대화,일반 대화


In [204]:
Sam['conversation'][id[0]]

'나 학교 오고 갈 때마다 버스 타고 전철 타고 대중교통 너무 빡세다\n나는 자취방 위치가 학교랑 그리 멀지 않아서 걸어가도 돼서 좋은데 불쌍하다\n아냐 그거 경차래 가 최고긴 한데 캐스퍼는 경차임\n가 일반 자동차보다 좋은 게 있어\n그래도 세단이 고급 지고 간지나잖아\n차박은 충분히 세단에서도 가능하고 보단 승용차나 세단이 나은 거 같아\n근데 요즘 차 크기보다는 전기차가 대세 아냐 엔진 소리도 작고\n길거리 돌아다니다 보면 난 전기차를 본 적이 없는데\n근데 나 어짜피 면허 없어서 차 못 탐 ㅎ\n그럼 여태 대중교통만 타고 여행을 다녔단 말이야\n대중교통이 최고지 1호선 타면 한여름에 감기 걸릴 수 있어\n출근 시간이나 퇴근 시간에 1호선 안 타봤으면 말을 하지 말아\n출퇴근 시간에 1호선 3호선 6호선 다 타봄\n타고 싶을 때 못 타고 내리고 싶을 때 못 내리잖아\n근데 또 대중교통은 가만히 앉아있으면 잘 가잖아\n근데 우린 서울인데 서울에선 차 있어도 힘들 거 같긴 해'

In [205]:
Sam['conversation'][id[1]]

'최근에 영화 본 거 있어 다들\n나 모가디슈 봤어 재밌더라고\n수영복 그녀 정말 제목부터 이상해\n이상한 거 많이 보고 다니네\n애니메이션은 나도 좀 봐 도라에몽 같은 거\n난 요 근래 나만의 없는 거리 라는 애니메이션을 봤어\n이 왜 나와 갑자기 진짜로\n나만의 없는 거리 에니메이션 추천 할께 재밌어\n나만의 없는 거리는 뭐 하는 영화야\n주인공이 18년 전 과거로 돌아가 연쇄 살인을 막는 이야기야 나도 아직 결말 까진 안 봤어\n오 대박이다 엄청 보고 싶어\n그치 나름 내용 좋다니까 명작이야\n난 영화 클래식을 최근에 자주 봐\n클래식 영화 이야\n그냥 사랑을 담은 영화야 꼭 봐 진짜로\n그래 그래 나도 로멘스 보고 눈물 짜는 거 좋아해'